In [47]:
import numpy as np

from fpt.data import join_face_df
from fpt.split import read_split
from fpt.path import DTFR, UTIL

In [56]:
face = join_face_df(DTFR, 'aihub_family')  # 16.1s


In [10]:
valid_uuids = read_split("valid")
x_valid = face.loc[valid_uuids]
x_valid = x_valid.reset_index().reset_index().set_index('uuid')
x_valid.keys()

Index(['index', 'key', 'label', 'image', 'category', 'option', 'data_category',
       'folder_name', 'family_id', 'personal_id', 'path', 'target'],
      dtype='object')

In [28]:
# Sample group
# x_valid.groupby('target').index.apply(list).to_frame().head()
# x_valid.groupby(['family_id', 'personal_id', 'category']).index.apply(list).to_frame().head(10)

In [ ]:
np.random.seed(22)
NUM_FOLDS = 10
CATEGORY = 'Family'
target_pair = f"pairs/pairs_{CATEGORY}.txt"

is_family = x_valid.category == CATEGORY
family_valid = x_valid[is_family]
idx_family_valid = family_valid.groupby('target').index.apply(list).to_frame()

with open(target_pair, "w") as f:
    for n in range(NUM_FOLDS):
        # matched
        matched_sample = idx_family_valid.sample(
            n=300, replace=False, random_state=n
        ).sort_values("target")
        for key, value in matched_sample.iterrows():
            idxs = value.loc["index"]
            selected = np.random.choice(idxs, size=2, replace=False)
            f.write(f"{key:8s}\t{selected[0]:4d}\t{selected[1]:4d}")
            f.write("\n")

        # mismatched
        for i in range(300):
            mismatched_sample = idx_family_valid.sample(
                n=2, replace=False, random_state=n * 1000 + i
            ).sort_values("target")
            sampled = [
                [key, np.random.choice(value.loc["index"], replace=False)]
                for key, value in mismatched_sample.iterrows()
            ]
            f.write(
                "{0:8s}\t{1:4s}\t{2:8s}\t{3:4s}".format(
                    *np.array(sampled).flatten().tolist()
                )
            )
            f.write("\n")